<a href="https://colab.research.google.com/github/Jahan08/Amber-tutorial/blob/main/Retrieve-Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

! pip install chembl_webresource_client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.2/55.2 KB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 KB 3.9 MB/s eta 0:00:00
  Attempting uninstall: itsdangerous
    Found existing installation: itsdangerous 1.1.0
    Uninstalling itsdangerous-1.1.0:
      Successfully uninstalled itsdangerous-1.1.0
  Attempting uninstall: attrs
    Found existing installation: attrs 22.2.0
    Uninstalling attrs-22.2.0:
      Successfully uninstalled attrs-22.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 1.1.4 requires itsdangerous<2.0,>=0.24, but you have itsdangerous 2.1.2 which is incompatible.


In [ ]:
import pandas as pd
from chembl_webresource_client.new_client import new_client

In [25]:
target = new_client.target
target_query = target.search('SARS-CoV')
targets = pd.DataFrame.from_dict(target_query)
targets

,cross_references,organism,pref_name,score,species_group_flag,target_chembl_id,target_components,target_type,tax_id
0,[],Severe acute respiratory syndrome-related coro...,SARS-CoV,32.0,False,CHEMBL4303836,[],ORGANISM,694009
1,[],Severe acute respiratory syndrome coronavirus 2,SARS-CoV-2,28.0,False,CHEMBL4303835,[],ORGANISM,2697049
2,[],SARS coronavirus,SARS coronavirus,15.0,False,CHEMBL612575,[],ORGANISM,227859
3,[],Homo sapiens,"Serine--tRNA ligase, cytoplasmic",14.0,False,CHEMBL4523232,"[{'accession': 'P49591', 'component_descriptio...",SINGLE PROTEIN,9606
4,"[{'xref_id': 'P0C6U8', 'xref_name': None, 'xre...",SARS coronavirus,SARS coronavirus 3C-like proteinase,11.0,False,CHEMBL3927,"[{'accession': 'P0C6U8', 'component_descriptio...",SINGLE PROTEIN,227859
5,"[{'xref_id': 'P0C6X7', 'xref_name': None, 'xre...",SARS coronavirus,Replicase polyprotein 1ab,4.0,False,CHEMBL5118,"[{'accession': 'P0C6X7', 'component_descriptio...",SINGLE PROTEIN,227859
6,[],Severe acute respiratory syndrome coronavirus 2,Replicase polyprotein 1ab,4.0,False,CHEMBL4523582,"[{'accession': 'P0DTD1', 'component_descriptio...",SINGLE PROTEIN,2697049


In [ ]:
df = pd.read_csv('/content/bioactivity_data_ACHN.csv', header = 0)

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (15,25,28,41) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [29]:
selected_target = targets.target_chembl_id[1]
selected_target

'CHEMBL4303835'

In [30]:
activity = new_client.activity
res = activity.filter(target_chembl_id=selected_target).filter(standard_type="Inhibition")

In [ ]:
df = pd.DataFrame.from_dict(res)
df

In [ ]:

# considering nM unit for MIC
df = df[(df['standard_units'] == 'nM')]
df = df[(df['relation'] == '=')]
len(df)

471

In [ ]:
# considering nM unit for MIC
df = df[(df['standard_units'] == '%')]
df = df[(df['relation'] == '=')]
len(df)

In [ ]:
fix = []

for i in df.standard_value:
    fix.append(float(i))

STATUS = []

for i in fix:
    if i < 20000:
        STATUS.append("Yes") #active
        
    elif 50000 <= i <= 10000:
        STATUS.append("No") #inactive
        
    else:
        STATUS.append("IN") #intermediate     

df = df.drop('value', 1)
df['value'] = fix
df['Activity'] = STATUS
df.head(3)

<ipython-input-33-20c913d0ecdd>:18: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df.drop('value', 1)


,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,...,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value,Activity
0,None,36426,[],CHEMBL619313,In vitro cytotoxicity against human tumor A498...,F,None,None,BAO_0000190,BAO_0000219,...,A498,9606,None,None,IC50,uM,UO_0000065,None,7500.0,Yes
1,None,44807,[],CHEMBL619313,In vitro cytotoxicity against human tumor A498...,F,None,None,BAO_0000190,BAO_0000219,...,A498,9606,None,None,IC50,uM,UO_0000065,None,53000.0,IN
2,None,49899,[],CHEMBL619313,In vitro cytotoxicity against human tumor A498...,F,None,None,BAO_0000190,BAO_0000219,...,A498,9606,None,None,IC50,uM,UO_0000065,None,63500.0,IN


In [ ]:
fix = []

for i in df.standard_value:
    fix.append(float(i))

STATUS = []

for i in fix:
    if i > 80:
        STATUS.append("Yes") #active
        
    elif i == 0:
        STATUS.append("No") #inactive
        
    else:
        STATUS.append("IN") #intermediate     

df = df.drop('value', 1)
df['value'] = fix
df['Activity'] = STATUS
df.head(3)

In [ ]:
# considering nM unit for MIC
df = df[(df['Activity'] != 'IN')]
len(df)

305

In [ ]:

df.to_csv('bioactivity_data_ACHN_IC50_nM.csv', index=False)
     